Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

In [3]:
combine_df = pd.read_csv('./combine_df.csv')
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when fatheare is dysfyounctional and is so sel...,"['when', 'fatheare', 'is', 'dysfyounctional', ...","['fatheare', 'dysfyounctional', 'selfish', 'da...","['fathear', 'dysfyounct', 'selfish', 'dareagar...","['fatheare', 'dysfyounctional', 'selfish', 'da..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks foare lyft careedit cannot youse cayous...,"['thanks', 'foare', 'lyft', 'careedit', 'can',...","['thanks', 'foare', 'lyft', 'careedit', 'youse...","['thank', 'foar', 'lyft', 'careedit', 'yous', ...","['thank', 'foare', 'lyft', 'careedit', 'youse'..."
2,3,0.0,bihday your majesty,bihday yoyouare majesty,"['bihday', 'yoyouare', 'majesty']","['bihday', 'yoyouare', 'majesty']","['bihday', 'yoyouar', 'majesti']","['bihday', 'yoyouare', 'majesty']"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in y...,"['model', 'love', 'you', 'take', 'with', 'you'...","['model', 'love', 'take', 'time', 'youare']","['model', 'love', 'take', 'time', 'youar']","['model', 'love', 'take', 'time', 'youare']"
4,5,0.0,factsguide: society now #motivation,factsgareinyouide society now motivation,"['factsgareinyouide', 'society', 'now', 'motiv...","['factsgareinyouide', 'society', 'motivation']","['factsgareinyouid', 'societi', 'motiv']","['factsgareinyouide', 'society', 'motivation']"


In [8]:
combine_df.shape

(49159, 8)

In [15]:
train_y = combine_df['label'][:21962]
train_y.tail()

21957    0.0
21958    0.0
21959    0.0
21960    0.0
21961    0.0
Name: label, dtype: float64

In [16]:
train_y.shape

(21962,)

In [17]:
train_x = combine_df['tweet_lemmatized'][:21962]
train_x.head()

0    ['fatheare', 'dysfyounctional', 'selfish', 'da...
1    ['thank', 'foare', 'lyft', 'careedit', 'youse'...
2                    ['bihday', 'yoyouare', 'majesty']
3          ['model', 'love', 'take', 'time', 'youare']
4       ['factsgareinyouide', 'society', 'motivation']
Name: tweet_lemmatized, dtype: object

In [18]:
valid_y = combine_df['label'][21962:31962]
valid_y.tail()

31957    0.0
31958    0.0
31959    0.0
31960    1.0
31961    0.0
Name: label, dtype: float64

In [19]:
valid_y.shape

(10000,)

In [20]:
valid_x = combine_df['tweet_lemmatized'][21962:31962]
valid_x.head()

21962    ['msyou', 'tareonline', 'love', 'tareI', 'seek...
21963    ['happen', 'see', 'pearefect', 'example', 'wom...
21964    ['hope', 'gareinyouy', 'say', 'come', 'tareyou...
21965          ['sex', 'blonde', 'sex', 'faarem', 'women']
21966    ['befoaree', 'yoyou', 'maybe', 'still', 'carey...
Name: tweet_lemmatized, dtype: object

In [21]:
valid_x.shape

(10000,)

In [13]:
test_x = combine_df['tweet_lemmatized'][31962:]
test_x.head()

31962    ['styoudiolife', 'aislife', 'areeqyouiarees', ...
31963    ['white', 'syoupareemacists', 'want', 'evearey...
31964    ['safe', 'ways', 'heal', 'yoyouare', 'acne', '...
31965    ['hp', 'see', 'yoyouaresed', 'chilateare', 'dy...
31966    ['ared', 'bihday', 'amazingarein', 'hilaareioy...
Name: tweet_lemmatized, dtype: object

In [14]:
test_x.shape

(17197,)

In [22]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [23]:
tokenizer = Tokenizer()

In [24]:
tokenizer.fit_on_texts(train_x)

In [25]:
sequences_train = tokenizer.texts_to_sequences(train_x)
sequences_val = tokenizer.texts_to_sequences(valid_x)
sequences_test = tokenizer.texts_to_sequences(test_x)

In [27]:
word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in train_x])

In [28]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [31]:
y_train = train_y
y_val = valid_y

In [29]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
39/39 [==============================] - 2s 63ms/step - loss: 0.3057 - accuracy: 0.9240 - val_loss: 0.2512 - val_accuracy: 0.9304
Epoch 2/10
39/39 [==============================] - 2s 57ms/step - loss: 0.2577 - accuracy: 0.9308 - val_loss: 0.2414 - val_accuracy: 0.9304
Epoch 3/10
39/39 [==============================] - 2s 55ms/step - loss: 0.1825 - accuracy: 0.9408 - val_loss: 0.1522 - val_accuracy: 0.9508
Epoch 4/10
39/39 [==============================] - 2s 62ms/step - loss: 0.0712 - accuracy: 0.9765 - val_loss: 0.1360 - val_accuracy: 0.9527
Epoch 5/10
39/39 [==============================] - 2s 60ms/step - loss: 0.0278 - accuracy: 0.9927 - val_loss: 0.1584 - val_accuracy: 0.9481


In [33]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 0s 20ms/step - loss: 0.1542 - accuracy: 0.9552


Test score: 0.15423792600631714
Test accuracy: 0.9552000164985657


In [34]:
model_1 = Sequential()

model_1.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_1.add(Masking(mask_value=0.0))
model_1.add(LSTM(64, recurrent_dropout=0.2))
model_1.add(Dense(64, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model_1.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
39/39 [==============================] - 5s 136ms/step - loss: 0.4083 - accuracy: 0.9154 - val_loss: 0.2486 - val_accuracy: 0.9304
Epoch 2/10
39/39 [==============================] - 5s 123ms/step - loss: 0.2392 - accuracy: 0.9308 - val_loss: 0.2076 - val_accuracy: 0.9304
Epoch 3/10
39/39 [==============================] - 5s 141ms/step - loss: 0.1428 - accuracy: 0.9471 - val_loss: 0.1342 - val_accuracy: 0.9545
Epoch 4/10
39/39 [==============================] - 5s 125ms/step - loss: 0.0661 - accuracy: 0.9785 - val_loss: 0.1505 - val_accuracy: 0.9586


In [35]:
score = model_1.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 1s 35ms/step - loss: 0.1531 - accuracy: 0.9555


Test score: 0.15307702124118805
Test accuracy: 0.9555000066757202


In [36]:
model_2 = Sequential()

model_2.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_2.add(Masking(mask_value=0.0))
model_2.add(GRU(64, recurrent_dropout=0.2))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model_2.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
39/39 [==============================] - 5s 117ms/step - loss: 0.4240 - accuracy: 0.9199 - val_loss: 0.2399 - val_accuracy: 0.9304
Epoch 2/10
39/39 [==============================] - 4s 106ms/step - loss: 0.2011 - accuracy: 0.9323 - val_loss: 0.1745 - val_accuracy: 0.9331
Epoch 3/10
39/39 [==============================] - 5s 121ms/step - loss: 0.1105 - accuracy: 0.9564 - val_loss: 0.1393 - val_accuracy: 0.9518
Epoch 4/10
39/39 [==============================] - 4s 107ms/step - loss: 0.0507 - accuracy: 0.9829 - val_loss: 0.1509 - val_accuracy: 0.9558


In [37]:
score = model_2.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 1s 39ms/step - loss: 0.1453 - accuracy: 0.9580


Test score: 0.14527447521686554
Test accuracy: 0.9580000042915344


Лучше всего себя показала последняя модель со слоем GRU.